<div style="text-align: center;">

# SVEUČILIŠTE U ZAGREBU
# FAKULTET ORGANIZACIJE I INFORMATIKE
# V A R A Ž D I N
<br><br><br><br><br><br>
**Ivan Simić  
Studij: Informacijski i poslovni sustavi**
<br><br><br><br><br><br><br><br><br><br><br><br><br>
# PREGLED DOKUMENATA STRANICE PREKO PYTHONA
# PROJEKT IZ KOLEGIJA PROGRAMIRANJE ZA ANALIZU PODATAKA
<br><br><br><br><br><br><br><br><br><br>
**Mentor/Mentorica:**  
**Prof. dr. sc. Markus Schatten**
<br><br><br><br><br><br><br><br>
**Varaždin, siječanj 2024.**

</div>

<br><br><br><br>

Ivan Simić

Izjava o izvornosti

*Izjavljujem da je moj projekt izvorni rezultat mojeg rada te da se u izradi istoga nisam koristio drugim izvorima osim onima koji su u njemu navedeni. Za izradu rada su korištene etički prikladne i prihvatljive metode i tehnike rada.*
**_______________________________________________________________________________________________________________________________**

<br><br><br><br>

# 1. Uvod

Kako se na web stranici mogu pronaći svi dokumenti? Kako odrediti samo dokumente u PDF formatu? Kako preuzeti te dokumente? Kako se upravlja bazom podataka? Kako se podaci preuzetih dokumenata mogu spremiti u bazu? Kako se može spojiti na kreiranu bazu i prikazati njene podatke?

Kako se sve ovo može izvesti u Pythonu?

Najbolji način za prikazati odgovore na ova pitanja je uz program koji će implementirati sve funkcije iz tih pitanja. Uz same odgovore na pitanja, također će se kreirati program koji pokazuje kako se podaci iz jednog formata mogu prebaciti u drugi format za lakše prikazivanje tih podataka.

<br><br><br><br>

# 2. Opis implementacije

Da ponovimo koje su sve funkcije potrebne u ovome programu:
1. Preuzimanje dokumenata određenog formata sa web stranice
2. Spremanje podataka tih dokumenta
3. Kreiranje baze podataka
4. Spremanje podataka dokumenta u kreiranu bazu
5. Kreiranje servisa za prikaz baze podataka
6. Pretraživanje podataka baze preko servisa

Za preuzimanje svih dokumenata sa web stranice potrebne su dvije biblioteke: "beautifulsoup4" i "requests". "beautifulsoup4" biblioteka omogućava programu da uzima podatke sa HTML stranice, a "requests" omogućava slanje HTTP zahtjeva. S prvom bibliotekom možemo pronaći sve dokumente sa stranice koje tražimo, a s drugom bibliotekom možemo preuzeti dokumente koje smo pronašli i njihove podatke [1][2].

Iz preuzetih dokumenata trebaju nam sljedeći podaci: naslov, putanja na računalu gdje je dokument preuzet, URL adresa s koje dokument dolazi, datum dokumenta i njegov sadržaj. Za traženje naslova i putanje dokumenta koristi se python modul "os" [9]. URL adresa može se pronaći preko "href" atributa dokumenta. Za uzimanje datuma potrebna će biti funkcija koja može pročitati datum iz zaglavlja dokumenta. Uzimanje sadržaja ovisi o strukturi dokumenta. Ako se radi o tekstualnom dokumentu, njegov sadržaj lako se može iščitati s "pdfminer" funkcijom "extract_text(ime_datoteke)" [10]. Ako se radi o skeniranom dokumentu postaje komplicirano. Kako bi se iz skeniranog PDF dokumenta mogao uzeti tekst prvo je potrebno taj dokument pretvoriti u sliku kako bi njime mogla upravljati "pytesseract" biblioteka. Ovo je moguće uz biblioteku "pdf2image" koja sadrži funkciju "convert_from_path" [12]. Kada se dokument prebaci u sliku, "pytesseract" može iščitati tekst uz pomoć funkcije "pytesseract.image_to_string" [11]. Također, potreban je poseban "pytesseract" modul koji omogućuje pravilno čitanje hrvatskog jezika iz slike.

Baza podataka s kojom se lako upravlja u Pythonu je "SQLite". Za manipuliranje baze potreban je kursor, s kojim možemo unositi spremljene podatke u bazu. Za umetanje i traženje podataka u bazi koriste se standardni SQL upiti. Za unos podataka iz dokumenata u bazu potrebno je svaki podatak zapisati u varijablu koja će se u atribut baze unijeti s "INSERT" upitom.

Na kraju, potrebno je podatke iz baze prikazati. Za ovu svrhu kreira se Flask server na localhost adresi. Također, treba dodati mogućnost pretraživanja tablice baze prema sadržaju, naslovu i datumu datoteka, što se može postići pozivanjem SQL upita na bazu podataka preko Flask servera. Zahtjev za upit može se poslati preko url-a.

<br><br><br><br>

# 3. Prikaz rada aplikacije

Za početak, potrebno je preuzeti sve biblioteke nužne za izvršavanje programa. No, kako bi "pytesseract" i "pdf2image" biblioteke radile pravilno, potrebno je na internetu pronaći i preuzeti dodatne materijale.

Tesseract installer: https://github.com/UB-Mannheim/tesseract/wiki  
Direktno preuzimanje: https://digi.bib.uni-mannheim.de/tesseract/tesseract-ocr-w64-setup-5.3.3.20231005.exe

Poppler: https://github.com/oschwartz10612/poppler-windows/releases/tag/v23.11.0-0  
Direktno preuzimanje: https://github.com/oschwartz10612/poppler-windows/releases/download/v23.11.0-0/Release-23.11.0-0.zip

Također, "pytesseract" treba dodatni modul s pomoću kojeg može čitati hrvatske znakove.  
Traineddata: https://github.com/tesseract-ocr/tessdoc/blob/main/Data-Files.md  
Direktno preuzimanje: https://github.com/tesseract-ocr/tessdata/raw/4.00/hrv.traineddata  
Ovaj modul treba staviti u direktorij "tessdata" unutar "Tesseract-OCR" instalacije na računalu [13].

In [10]:
!pip install requests
!pip install beautifulsoup4
!pip install pdfminer.six
!pip install pdf2image
!pip install pytesseract
!pip install flask

  Obtaining dependency information for pdf2image from https://files.pythonhosted.org/packages/62/33/61766ae033518957f877ab246f87ca30a85b778ebaad65b7f74fa7e52988/pdf2image-1.17.0-py3-none-any.whl.metadata


<br><br>

## 3.1. Preuzimanje dokumenata i kreiranje baze
<br>
Na početku postavljamo URL stranice s koje preuzimamo dokumente. Na adresu šaljemo GET zahtjev. Ako zahtjev ne uspije, program vraća poruku greške. Ako zahtjev uspije, prolazimo kroz cijelu stranicu i tražimo sve poveznice. Tu se kreira direktorij "preuzeti_dokumenti" na lokalnom računalu u kojeg će se svi dokumenti spremiti [6], te se kreira baza "SQLdokumenti.db" s jednom tablicom "dokument" u koju ćemo spremiti sve podatke preuzetih dokumenata [7].

Sada možemo proći kroz sve pronađene poveznice. Gledamo samo one poveznice koje završavaju s ".pdf" što označava da poveznica sadrži dokument u formatu koji tražimo. Za preuzimanje PDF dokumenta, te za uzimanje njegovog datuma i sadržaja potrebne su posebne funkcije.

Preuzimanje se odvija u funkciji "file_preuzmi". Svaki dokument je zapisan u obliku "direktorij\ime_dokumenta" kako bi se dokumenti mogli spremiti u odabrani direktorij. Provjerava se da li već postoji dokument s istim imenom u direktoriju, te se dokument preskače ako postoji. Na putanju dokumenta šalje se GET zahtjev kako bi se dokument mogao spremiti na računalo korištenjem "file.write(odgovor_zahtjeva)" funkcije [3][4].

Datum se uzima na sličan način. Ovaj put se uzima zaglavlje dokumenta, te se u zaglavlju traži atribut "Last-Modified" koji sadrži datum dokumenta. Varijabla u koju je stavljen taj atribut postavlja se u traženi format kako bi se datum mogao pravilno prikazati u bazi [5].

Sada slijedi sadržaj. Prvo će se pokušati uzeti tekst iz dokumenta korištenjem "pdfminer" funkcije "extract_text(file_ime)". Ako ovim putem sadržaj ostane prazan ("\f") znači da se radi o skeniranom PDF dokumentu. Dakle, potrebno je izvesti OCR, tj. optičko prepoznavanje znakova (eng. *optical character recognition*). Prvo treba dokument pretvoriti u sliku za OCR. Ovo je moguće s "convert_from_path" funkcijom iz "pdf2image" biblioteke [12]. Nakon toga treba maknuti prazan znak iz sadržaja. Sada se, za svaku stranicu dokumenta, provodi OCR s funkcijom "pytesseract.image_to_string". U funkciji je potrebno zadati jezik na hrvatski. NAPOMENA: kako bi "pytesseract" i "pdf2image" biblioteke funkcionirale pravilno u kodu je potrebno zadati putanje do dodatno preuzetih materijala na računalu. Obje putanje u kodu imaju zapisan komentar sa uputama.

Sada imamo sve podatke potrebne da popunimo bazu. Stvorimo varijablu za svaki podatak. Sadržaj svake varijable postavlja se u atribut s istim imenom u tablici. Za svaki preuzeti dokument prikazuje se poruka uspješnog preuzimanja. Nakon zadnjeg preuzetog dokumenta prikazuje se završna poruka koja označava kraj rada.

In [2]:
import os
import sqlite3
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from pdfminer.high_level import extract_text
from pdf2image import convert_from_path
import pytesseract

#Postaviti putanju gdje se nalazi "tesseract.exe" u "tesseract" instalaciji na računalu
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\simic\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'

#Funkcija za preuzimanje
def file_preuzmi(url_absolute, file_direktorij):
    file_ime = os.path.join(file_direktorij, os.path.basename(url_absolute))
    if os.path.exists(file_ime):
        return None
    response = requests.get(url_absolute)
    #'wb' predstavlja način otvaranja dokumenta ("binary write mode")
    with open(file_ime, 'wb') as file:
        file.write(response.content)
    return file_ime

#Funkcija za uzimanje datuma iz dokumenta
def uzmi_file_datum(url_absolute):
    response = requests.head(url_absolute)
    last_modified = response.headers.get('Last-Modified')
    if last_modified:
        file_datum = datetime.strptime(last_modified, '%a, %d %b %Y %H:%M:%S %Z')
        return file_datum    
    return None

# Funkcija za ekstrakciju teksta iz dokumenta
def uzmi_file_sadrzaj(file_ime):
    file_sadrzaj = extract_text(file_ime)
    if file_sadrzaj == "\f":
        #Postaviti putanju gdje se nalazi "bin" direktorij u "poppler" instalaciji na računalu
        konverzija = convert_from_path(file_ime, poppler_path=r'C:\Users\simic\Downloads\Release-23.11.0-0\poppler-23.11.0\Library\bin')
        file_sadrzaj = ""
        for slika in konverzija:
            file_sadrzaj += (pytesseract.image_to_string(slika, lang='hrv') + "\n")
        return file_sadrzaj
    else:
        return file_sadrzaj

#Početak programa
url = "https://www.foi.unizg.hr/hr/dokumenti"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    linkovi = soup.find_all('a')
    file_direktorij = 'preuzeti_dokumenti'
    os.makedirs(file_direktorij, exist_ok=True)
    
    #Kreiraj SQLite bazu, nova se kreira ako već ne postoji
    con = sqlite3.connect('SQLdokumenti.db')
    cursor = con.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS dokument (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            naslov TEXT,
            putanja TEXT,
            url TEXT,
            datum TEXT,
            sadrzaj TEXT
        )
    ''')
    con.commit()
    
    #Prolaženje kroz sve poveznice i traženje pdf dokumenata
    for link in linkovi:
        url_absolute = link.get('href') 
        if url_absolute and url_absolute.endswith('.pdf'):
            file_ime = file_preuzmi(url_absolute, file_direktorij)
            if file_ime:
                file_datum = uzmi_file_datum(url_absolute)
                file_sadrzaj = uzmi_file_sadrzaj(file_ime)
                
                #Spremanje podataka o dokumentu
                naslov = os.path.basename(file_ime)
                putanja = os.path.abspath(file_ime)
                url = url_absolute
                datum = file_datum if file_datum else datetime.today().strftime('%Y-%m-%d %H:%M:%S')
                sadrzaj = file_sadrzaj
                #Unos podataka u atribute tablice u bazi
                cursor.execute('''
                    INSERT INTO dokument (naslov, putanja, url, datum, sadrzaj)
                    VALUES (?, ?, ?, ?, ?)
                ''', (naslov, putanja, url, datum, sadrzaj))
                con.commit()
                print(f"Dokument preuzet: {os.path.basename(file_ime)}")
    con.close()
    print("Preuzimanje gotovo. Baza kreirana.")
else:
    print(f"Neuspjelo preuzimanje.")

Dokument preuzet: odluka_o_izmjeni_odluke_pok_2023_10_12_0.pdf
Dokument preuzet: odluka-izvanredni_izbori_za_studentski_zbor-sijecanj_2024.pdf
Dokument preuzet: fv_dnevni_red_2023-12-07_3._sjednica.pdf
Dokument preuzet: politika-privatnosti-foi.pdf
Dokument preuzet: popis_gs_u_sukobu_interesa_2023.pdf
Dokument preuzet: fv_zakljucci_2023-11-16_2._sjednica.pdf
Preuzimanje gotovo. Baza kreirana.


<br><br>

## 3.2. Postavljanje Flask servera

Sada možemo postaviti REST API za pristup bazi podataka. To ćemo postići s kreiranjem Flask servera na localhost adresi. Prvo napravimo malu funkciju s kojom se spaja na bazu podataka, pošto će se za svako pretraživanje podataka preko servera prvo biti potrebno spojiti na bazu. Početna (eng. *index*) stranica služit će samo za prikazivanje svih funkcionalnosti i kako doći do njih preko URL-a.

Prikaz svih dokumenata može se izvesti tako da se izvede SQL upit na bazu kojim će se vratiti svi podaci iz tablice "dokument" [7]. Pretraživanje sadržaja, naslova i datuma se izvodi na sličan način. Traženi će atribut u upitima za svako pretraživanje biti drugačiji. Tu će također biti potrebno poslati upit (eng. *query*) preko URL-a kako bi mogli pretražiti sadržaj tablice. Pošto se pretraživanje datuma izvodi preko raspona datuma, pretraživanje će tražiti dva upita; "pocetak" i "kraj" [8]. Ako se u URL ne unese upit u pravilnom formatu za pretraživanje atributa prikazat će se greška u kojoj je opisan pravilan način za unos upita. Sve poruke će se prikazivati u json formatu.

In [1]:
import threading
from flask import Flask, jsonify, request
import sqlite3

app = Flask(__name__)
port = "5000"

#Funkcija za spajanje na bazu za skraćeno pozivanje
def spajanje():
    return sqlite3.connect('SQLdokumenti.db')

#Početna stranica
@app.route("/")
def index():
    poruka = (
        "Dobro došli! Za pregled svih dokumenata upišite: /dokumenti. "
        "Za pretraživanje sadržaja dokumenata, upišite: /dokumenti/sadrzaj. "
        "Za pretraživanje naslova dokumenata, upišite: /dokumenti/naslov. "
        "Za pretraživanje dokumenata po datumu, upišite: /dokumenti/datum."
    )
    return jsonify(poruka)

#Izlistavanje svih dokumenata iz baze
@app.route('/dokumenti', methods=['GET'])
def uzmi_dokumente():
    con = spajanje()
    cursor = con.cursor()
    cursor.execute('SELECT * FROM dokument')
    dokumenti = cursor.fetchall()
    con.close()
    return jsonify({'Lista dokumenata': dokumenti})

#Pretraživanje atributa "sadrzaj"
@app.route('/dokumenti/sadrzaj', methods=['GET'])
def nadji_sadrzaj():
    query = request.args.get('query')
    if not query:
        return jsonify({'ERROR': 'Potreban je query. Format: /dokumenti/sadrzaj?query=trazeni_sadrzaj'}), 400
    
    con = spajanje()
    cursor = con.cursor()
    cursor.execute('SELECT * FROM dokument WHERE sadrzaj LIKE ?', (f"%{query}%",))
    dokumenti = cursor.fetchall()
    con.close()
    return jsonify({'Pronadjeno': dokumenti})

#Pretraživanje atributa "naslov"
@app.route('/dokumenti/naslov', methods=['GET'])
def nadji_naslov():
    query = request.args.get('query')
    if not query:
        return jsonify({'ERROR': 'Potreban je query. Format: /dokumenti/naslov?query=trazeni_naslov'}), 400
    
    con = spajanje()
    cursor = con.cursor()
    cursor.execute('SELECT * FROM dokument WHERE naslov LIKE ?', (f"%{query}%",))
    dokumenti = cursor.fetchall()
    con.close()
    return jsonify({'Pronadjeno': dokumenti})

#Pretraživanje atributa "datum"
@app.route('/dokumenti/datum', methods=['GET'])
def nadji_datum():
    pocetak = request.args.get('pocetak')
    kraj = request.args.get('kraj')
    if not pocetak or not kraj:
        return jsonify({'ERROR': 'Potrebni datumi. Format: /dokumenti/datum?pocetak=trazeni_datum&kraj=trazeni_datum'}), 400
    
    con = spajanje()
    cursor = con.cursor()
    cursor.execute('SELECT * FROM dokument WHERE datum BETWEEN ? AND ?', (pocetak, kraj))
    dokumenti = cursor.fetchall()
    con.close()
    return jsonify({'Pronadjeno': dokumenti})

#Pokretanje Flask servera
threading.Thread(target=app.run, kwargs={"host": "127.0.0.1", "port": port, "use_reloader": False}).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Jan/2024 21:03:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jan/2024 21:03:21] "GET /dokumenti HTTP/1.1" 200 -
127.0.0.1 - - [10/Jan/2024 21:03:29] "GET /dokumenti/sadrzaj?query=foi HTTP/1.1" 200 -
127.0.0.1 - - [10/Jan/2024 21:03:34] "GET /dokumenti/naslo?query=foi HTTP/1.1" 404 -
127.0.0.1 - - [10/Jan/2024 21:03:34] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Jan/2024 21:03:37] "GET /dokumenti/naslov?query=foi HTTP/1.1" 200 -
127.0.0.1 - - [10/Jan/2024 21:03:41] "GET /dokumenti/naslov?query=odluk HTTP/1.1" 200 -
127.0.0.1 - - [10/Jan/2024 21:04:18] "GET /dokumenti/naslov?datum?pocetak=2023-11-20&kraj=2023-12-20 HTTP/1.1" 400 -
127.0.0.1 - - [10/Jan/2024 21:05:09] "GET /dokumenti/datum?pocetak=2023-11-24&kraj=2023-12-08 HTTP/1.1" 200 -


<br><br><br><br>

# 4. Kritički osvrt

Ovaj program bi mogao ubrzati pretraživanje podataka ako su traženi podaci smješteni u PDF dokumentima na nekoj web stranici. Korisnicima bi program omogućio da jednim potezom preuzmu sve dokumente, te da njihov sadržaj pregledaju i pretraže na svojem serveru.

Glavno ograničenje ovog programa je to što se sam sadržaj baze može vidjeti tek na pokrenutom Flask serveru, tj. ne postoji brži i pristupačniji način u ovom programu koji omogućuje da se pregledaju podaci preuzetih dokumenata.

<br><br><br><br>

# 5. Zaključak

S implementacijom ovog programa dobiveni su odgovori na sva pitanja postavljena u uvodu. Sada imamo program kojim je moguće pronaći i preuzeti sve PDF dokumente sa odabrane web stranice, te pregledati podatke preuzetih dokumenata na vlastitom serveru. Naravno, treba pripaziti na to da se dokumenti preuzimaju sa web stranice koja ima javno dostupne dokumente kako ne bi došlo do potencijalnog kršenja autorskih prava.

<br><br><br><br>

# 6. Literatura

[1]	Beautiful Soup (bez dat.) "Beautiful Soup Documentation“ [Na internetu]. Dostupno: https://beautiful-soup-4.readthedocs.io/en/latest/ [pristupano 3.1.2023.].

[2]	PyPi (bez dat.) "Project description: Requests“ [Na internetu]. Dostupno: https://pypi.org/project/requests/ [pristupano 3.1.2023.].

[3]	GeeksforGeeks (bez dat.) "Downloading PDFs with Python using Requests and BeautifulSoup“ [Na internetu]. Dostupno: https://www.geeksforgeeks.org/downloading-pdfs-with-python-using-requests-and-beautifulsoup/ [pristupano 3.1.2023.].

[4]	stack overflow (bez dat.) "How to download pdf files using Python?“ [Na internetu]. Dostupno: https://stackoverflow.com/questions/43892092/how-to-download-pdf-files-using-python [pristupano 3.1.2023.].

[5]	stack overflow (bez dat.) "Python. Get file date and time (timestamp) from HTTP server“ [Na internetu]. Dostupno: https://stackoverflow.com/questions/58811813/python-get-file-date-and-time-timestamp-from-http-server [pristupano 3.1.2023.].

[6]	GeeksforGeeks (bez dat.) "Create a directory in Python“ [Na internetu]. Dostupno: https://www.geeksforgeeks.org/create-a-directory-in-python/  [pristupano 3.1.2023.].

[7]	python (bez dat.) "sqlite3 — DB-API 2.0 interface for SQLite databases“ [Na internetu]. Dostupno: https://docs.python.org/3/library/sqlite3.html [pristupano 3.1.2023.].

[8]	J. V. Schooneveld , "Python and REST APIs: Interacting With Web Services" (bez dat.) [Na internetu]. Dostupno: https://realpython.com/api-integration-in-python/ [pristupano 27.12.2023.].

[9]	GeeksforGeeks (bez dat.) "OS Path module in Python“ [Na internetu]. Dostupno: https://www.geeksforgeeks.org/os-path-module-python/ [pristupano 3.1.2023.].

[10] pdfminer.six (bez dat.) "Extract text from a PDF using Python“ [Na internetu]. Dostupno: https://pdfminersix.readthedocs.io/en/latest/tutorial/highlevel.html [pristupano 3.1.2023.].

[11] PyPi (bez dat.) "pytesseract 0.3.10“ [Na internetu]. Dostupno: https://pypi.org/project/pytesseract/ [pristupano 4.1.2023.].

[12] PyPi (bez dat.) "pdf2image 1.17.0“ [Na internetu]. Dostupno: https://pypi.org/project/pdf2image/ [pristupano 4.1.2023.].

[13] stack overflow (bez dat.) "How do I install a new language pack for Tesseract on Windows“ [Na internetu]. Dostupno: https://stackoverflow.com/questions/63048908/how-do-i-install-a-new-language-pack-for-tesseract-on-windows [pristupano 4.1.2023.].

[14] OpenAI (bez dat.) "ChatGPT“ [Alat]. Dostupno: https://openai.com/chatgpt [pristupano 3.1.2023.].